### カスタム評価関数を使用して、last入りデータのselectkbestで実行

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import pandas_profiling as pp

book_name = '601'

warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)



In [11]:
%%time
h5File = "temp_data/h5/prepared_data_with_last.h5";
key_train = 'prepared_01/train';
key_valid = 'prepared_01/valid';
key_test = 'prepared_01/test';

train = pd.read_hdf(h5File, key_train); gc.collect();
valid = pd.read_hdf(h5File, key_valid); gc.collect();
test = pd.read_hdf(h5File, key_test); gc.collect();


CPU times: user 79.9 ms, sys: 421 ms, total: 501 ms
Wall time: 839 ms


In [12]:
train

,count,S_2_days,P_2_min,P_2_max,P_2_avg,D_39_min,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,B_2_avg,R_1_min,R_1_max,R_1_avg,S_3_min,S_3_max,S_3_avg,D_41_min,D_41_max,D_41_avg,B_3_min,B_3_max,B_3_avg,D_42_min,D_42_max,D_42_avg,D_43_min,D_43_max,D_43_avg,D_44_min,D_44_max,D_44_avg,B_4_min,B_4_max,B_4_avg,D_45_min,D_45_max,D_45_avg,B_5_min,B_5_max,B_5_avg,R_2_min,R_2_max,R_2_avg,D_46_min,D_46_max,D_46_avg,D_47_min,D_47_max,D_47_avg,D_48_min,D_48_max,D_48_avg,D_49_min,D_49_max,D_49_avg,B_6_min,B_6_max,B_6_avg,B_7_min,B_7_max,B_7_avg,B_8_min,B_8_max,B_8_avg,D_50_min,D_50_max,D_50_avg,D_51_min,D_51_max,D_51_avg,B_9_min,B_9_max,B_9_avg,R_3_min,R_3_max,R_3_avg,D_52_min,D_52_max,D_52_avg,P_3_min,P_3_max,P_3_avg,B_10_min,B_10_max,B_10_avg,D_53_min,D_53_max,D_53_avg,S_5_min,S_5_max,S_5_avg,B_11_min,B_11_max,B_11_avg,S_6_min,S_6_max,S_6_avg,D_54_min,D_54_max,D_54_avg,R_4_min,R_4_max,R_4_avg,S_7_min,S_7_max,S_7_avg,B_12_min,B_12_max,B_12_avg,S_8_min,S_8_max,S_8_avg,D_55_min,D_55_max,D_55_avg,D_56_min,D_56_max,D_56_avg,B_13_min,B_13_max,B_13_avg,R_5_min,R_5_max,R_5_avg,D_58_min,D_58_max,D_58_avg,S_9_min,S_9_max,S_9_avg,B_14_min,B_14_max,B_14_avg,D_59_min,D_59_max,D_59_avg,D_60_min,D_60_max,D_60_avg,D_61_min,D_61_max,D_61_avg,B_15_min,B_15_max,B_15_avg,S_11_min,S_11_max,S_11_avg,D_62_min,D_62_max,D_62_avg,D_63_cnt_null,D_63_cnt_CL,D_63_cnt_CO,D_63_cnt_CR,D_63_cnt_XL,D_63_cnt_XM,D_63_cnt_XZ,D_64_cnt_null,D_64_cnt_m1,D_64_cnt_O,D_64_cnt_R,D_64_cnt_U,D_65_min,D_65_max,D_65_avg,B_16_min,B_16_max,B_16_avg,B_17_min,B_17_max,B_17_avg,B_18_min,B_18_max,B_18_avg,B_19_min,B_19_max,B_19_avg,D_66_cnt_null,D_66_cnt_0,D_66_cnt_1,B_20_min,B_20_max,B_20_avg,D_68_cnt_null,D_68_cnt_0,D_68_cnt_1,D_68_cnt_2,D_68_cnt_3,D_68_cnt_4,D_68_cnt_5,D_68_cnt_6,S_12_min,S_12_max,S_12_avg,R_6_min,R_6_max,R_6_avg,S_13_min,S_13_max,S_13_avg,B_21_min,B_21_max,B_21_avg,D_69_min,D_69_max,D_69_avg,B_22_min,B_22_max,B_22_avg,D_70_min,D_70_max,D_70_avg,D_71_min,D_71_max,D_71_avg,D_72_min,D_72_max,D_72_avg,S_15_min,S_15_max,S_15_avg,B_23_min,B_23_max,B_23_avg,D_73_min,D_73_max,D_73_avg,P_4_min,P_4_max,P_4_avg,D_74_min,D_74_max,D_74_avg,D_75_min,D_75_max,D_75_avg,D_76_min,D_76_max,D_76_avg,B_24_min,B_24_max,B_24_avg,R_7_min,R_7_max,R_7_avg,...,B_31_avg,S_19_min,S_19_max,S_19_avg,R_19_min,R_19_max,R_19_avg,B_32_min,B_32_max,B_32_avg,S_20_min,S_20_max,S_20_avg,R_20_min,R_20_max,R_20_avg,R_21_min,R_21_max,R_21_avg,B_33_min,B_33_max,B_33_avg,D_89_min,D_89_max,D_89_avg,R_22_min,R_22_max,R_22_avg,R_23_min,R_23_max,R_23_avg,D_91_min,D_91_max,D_91_avg,D_92_min,D_92_max,D_92_avg,D_93_min,D_93_max,D_93_avg,D_94_min,D_94_max,D_94_avg,R_24_min,R_24_max,R_24_avg,R_25_min,R_25_max,R_25_avg,D_96_min,D_96_max,D_96_avg,S_22_min,S_22_max,S_22_avg,S_23_min,S_23_max,S_23_avg,S_24_min,S_24_max,S_24_avg,S_25_min,S_25_max,S_25_avg,S_26_min,S_26_max,S_26_avg,D_102_min,D_102_max,D_102_avg,D_103_min,D_103_max,D_103_avg,D_104_min,D_104_max,D_104_avg,D_105_min,D_105_max,D_105_avg,D_106_min,D_106_max,D_106_avg,D_107_min,D_107_max,D_107_avg,B_36_min,B_36_max,B_36_avg,B_37_min,B_37_max,B_37_avg,R_26_min,R_26_max,R_26_avg,R_27_min,R_27_max,R_27_avg,B_38_cnt_null,B_38_cnt_0,B_38_cnt_1,B_38_cnt_2,B_38_cnt_3,B_38_cnt_4,B_38_cnt_5,B_38_cnt_6,B_38_cnt_7,D_108_min,D_108_max,D_108_avg,D_109_min,D_109_max,D_109_avg,D_110_min,D_110_max,D_110_avg,D_111_min,D_111_max,D_111_avg,B_39_min,B_39_max,B_39_avg,D_112_min,D_112_max,D_112_avg,B_40_min,B_40_max,B_40_avg,S_27_min,S_27_max,S_27_avg,D_113_min,D_113_max,D_113_avg,D_114_cnt_null,D_114_cnt_0,D_114_cnt_1,D_115_min,D_115_max,D_115_avg,D_116_cnt_null,D_116_cnt_0,D_116_cnt_1,D_117_cnt_null,D_117_cnt_m1,D_117_cnt_1,D_117_cnt_2,D_117_cnt_3,D_117_cnt_4,D_117_cnt_5,D_117_cnt_6,D_118_min,D_118_max,D_118_avg,D_119_min,D_119_max,D_119_avg,D_120_cnt_null,D_120_cnt_0,D_120_cnt_1,D_121_min,D_121_max,D_121_avg,D_122_min,D_122_max,D_122_avg,D_123_min,D_123_max,D_123_avg,D_124_min,D_124_max,D_124_avg,D_125_min,D_125_max,D_125_avg,D_126_cnt_null,D_126_cnt_m1,D_126_cnt_0,D_126_c

In [13]:
test

,count,S_2_days,P_2_min,P_2_max,P_2_avg,D_39_min,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,B_2_avg,R_1_min,R_1_max,R_1_avg,S_3_min,S_3_max,S_3_avg,D_41_min,D_41_max,D_41_avg,B_3_min,B_3_max,B_3_avg,D_42_min,D_42_max,D_42_avg,D_43_min,D_43_max,D_43_avg,D_44_min,D_44_max,D_44_avg,B_4_min,B_4_max,B_4_avg,D_45_min,D_45_max,D_45_avg,B_5_min,B_5_max,B_5_avg,R_2_min,R_2_max,R_2_avg,D_46_min,D_46_max,D_46_avg,D_47_min,D_47_max,D_47_avg,D_48_min,D_48_max,D_48_avg,D_49_min,D_49_max,D_49_avg,B_6_min,B_6_max,B_6_avg,B_7_min,B_7_max,B_7_avg,B_8_min,B_8_max,B_8_avg,D_50_min,D_50_max,D_50_avg,D_51_min,D_51_max,D_51_avg,B_9_min,B_9_max,B_9_avg,R_3_min,R_3_max,R_3_avg,D_52_min,D_52_max,D_52_avg,P_3_min,P_3_max,P_3_avg,B_10_min,B_10_max,B_10_avg,D_53_min,D_53_max,D_53_avg,S_5_min,S_5_max,S_5_avg,B_11_min,B_11_max,B_11_avg,S_6_min,S_6_max,S_6_avg,D_54_min,D_54_max,D_54_avg,R_4_min,R_4_max,R_4_avg,S_7_min,S_7_max,S_7_avg,B_12_min,B_12_max,B_12_avg,S_8_min,S_8_max,S_8_avg,D_55_min,D_55_max,D_55_avg,D_56_min,D_56_max,D_56_avg,B_13_min,B_13_max,B_13_avg,R_5_min,R_5_max,R_5_avg,D_58_min,D_58_max,D_58_avg,S_9_min,S_9_max,S_9_avg,B_14_min,B_14_max,B_14_avg,D_59_min,D_59_max,D_59_avg,D_60_min,D_60_max,D_60_avg,D_61_min,D_61_max,D_61_avg,B_15_min,B_15_max,B_15_avg,S_11_min,S_11_max,S_11_avg,D_62_min,D_62_max,D_62_avg,D_63_cnt_null,D_63_cnt_CL,D_63_cnt_CO,D_63_cnt_CR,D_63_cnt_XL,D_63_cnt_XM,D_63_cnt_XZ,D_64_cnt_null,D_64_cnt_m1,D_64_cnt_O,D_64_cnt_R,D_64_cnt_U,D_65_min,D_65_max,D_65_avg,B_16_min,B_16_max,B_16_avg,B_17_min,B_17_max,B_17_avg,B_18_min,B_18_max,B_18_avg,B_19_min,B_19_max,B_19_avg,D_66_cnt_null,D_66_cnt_0,D_66_cnt_1,B_20_min,B_20_max,B_20_avg,D_68_cnt_null,D_68_cnt_0,D_68_cnt_1,D_68_cnt_2,D_68_cnt_3,D_68_cnt_4,D_68_cnt_5,D_68_cnt_6,S_12_min,S_12_max,S_12_avg,R_6_min,R_6_max,R_6_avg,S_13_min,S_13_max,S_13_avg,B_21_min,B_21_max,B_21_avg,D_69_min,D_69_max,D_69_avg,B_22_min,B_22_max,B_22_avg,D_70_min,D_70_max,D_70_avg,D_71_min,D_71_max,D_71_avg,D_72_min,D_72_max,D_72_avg,S_15_min,S_15_max,S_15_avg,B_23_min,B_23_max,B_23_avg,D_73_min,D_73_max,D_73_avg,P_4_min,P_4_max,P_4_avg,D_74_min,D_74_max,D_74_avg,D_75_min,D_75_max,D_75_avg,D_76_min,D_76_max,D_76_avg,B_24_min,B_24_max,B_24_avg,R_7_min,R_7_max,R_7_avg,...,B_31_avg,S_19_min,S_19_max,S_19_avg,R_19_min,R_19_max,R_19_avg,B_32_min,B_32_max,B_32_avg,S_20_min,S_20_max,S_20_avg,R_20_min,R_20_max,R_20_avg,R_21_min,R_21_max,R_21_avg,B_33_min,B_33_max,B_33_avg,D_89_min,D_89_max,D_89_avg,R_22_min,R_22_max,R_22_avg,R_23_min,R_23_max,R_23_avg,D_91_min,D_91_max,D_91_avg,D_92_min,D_92_max,D_92_avg,D_93_min,D_93_max,D_93_avg,D_94_min,D_94_max,D_94_avg,R_24_min,R_24_max,R_24_avg,R_25_min,R_25_max,R_25_avg,D_96_min,D_96_max,D_96_avg,S_22_min,S_22_max,S_22_avg,S_23_min,S_23_max,S_23_avg,S_24_min,S_24_max,S_24_avg,S_25_min,S_25_max,S_25_avg,S_26_min,S_26_max,S_26_avg,D_102_min,D_102_max,D_102_avg,D_103_min,D_103_max,D_103_avg,D_104_min,D_104_max,D_104_avg,D_105_min,D_105_max,D_105_avg,D_106_min,D_106_max,D_106_avg,D_107_min,D_107_max,D_107_avg,B_36_min,B_36_max,B_36_avg,B_37_min,B_37_max,B_37_avg,R_26_min,R_26_max,R_26_avg,R_27_min,R_27_max,R_27_avg,B_38_cnt_null,B_38_cnt_0,B_38_cnt_1,B_38_cnt_2,B_38_cnt_3,B_38_cnt_4,B_38_cnt_5,B_38_cnt_6,B_38_cnt_7,D_108_min,D_108_max,D_108_avg,D_109_min,D_109_max,D_109_avg,D_110_min,D_110_max,D_110_avg,D_111_min,D_111_max,D_111_avg,B_39_min,B_39_max,B_39_avg,D_112_min,D_112_max,D_112_avg,B_40_min,B_40_max,B_40_avg,S_27_min,S_27_max,S_27_avg,D_113_min,D_113_max,D_113_avg,D_114_cnt_null,D_114_cnt_0,D_114_cnt_1,D_115_min,D_115_max,D_115_avg,D_116_cnt_null,D_116_cnt_0,D_116_cnt_1,D_117_cnt_null,D_117_cnt_m1,D_117_cnt_1,D_117_cnt_2,D_117_cnt_3,D_117_cnt_4,D_117_cnt_5,D_117_cnt_6,D_118_min,D_118_max,D_118_avg,D_119_min,D_119_max,D_119_avg,D_120_cnt_null,D_120_cnt_0,D_120_cnt_1,D_121_min,D_121_max,D_121_avg,D_122_min,D_122_max,D_122_avg,D_123_min,D_123_max,D_123_avg,D_124_min,D_124_max,D_124_avg,D_125_min,D_125_max,D_125_avg,D_126_cnt_null,D_126_cnt_m1,D_126_cnt_0,D_126_c

In [14]:
valid


customer_ID
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5    0.0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1    0.0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc    0.0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed    0.0
                                                                   ... 
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7    0.0
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286    0.0
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6    0.0
ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf388145b2c3d01967fcce461    1.0
fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eaba8b115f71cab04681    0.0
Name: target, Length: 458913, dtype: float64

In [15]:
X_train = train
y_train = valid
X_test = test

In [ ]:
gc.collect()

In [21]:
import lightgbm as lgb
from typing import Tuple

def amex_metric(preds: np.ndarray, data: lgb.Dataset) -> Tuple[str, float, bool]:
# def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
#     print('preds', preds)
#     print('preds.shape', preds.shape)
#     print('data.get_label()', data.get_label())
#     print('data.get_label().shape', data.get_label().shape)

    y_pred = pd.DataFrame({'prediction': pd.Series(preds)})
    y_true = pd.DataFrame({'target': pd.Series(data.get_label())})

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns').sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns').sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

#     df_y_true = 
    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)
    m = 0.5 * (g + d)

    return 'amex_metric', m, True


In [22]:
%%time
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
# skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=0)

params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40,
    "metric" : "None",  # <= set `None` for custom metric
#     'n_jobs': 4
}


for fold_id, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    X_tr , X_val = X_train.iloc[train_index], X_train.iloc[test_index]
    y_tr , y_val = y_train.iloc[train_index], y_train.iloc[test_index]

    print("============== fold_id:", fold_id)

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    print("--- train ---")
    %time model = lgb.train(params, lgb_train,\
        valid_sets=[lgb_train, lgb_eval],\
        verbose_eval=10,\
        num_boost_round=1000,\
        early_stopping_rounds=10,\
        feval=amex_metric) # custom metrics

    print("--- oof_train ---")
    %time oof_train[test_index] = model.predict(X_val, num_iteration=model.best_iteration)
    print("--- y_pred ---")
    %time y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)


============== fold_id: 0
--- train ---
[LightGBM] [Info] Number of positive: 95062, number of negative: 272068
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.664812 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 158466
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 581
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051523
[LightGBM] [Info] Start training from score -1.051523
Training until validation scores don't improve for 10 rounds
[10]	training's amex_metric: 0.709695	valid_1's amex_metric: 0.70443
[20]	training's amex_metric: 0.721444	valid_1's amex_metric: 0.715509
[30]	training's amex_metric: 0.729882	valid_1's amex_metric: 0.723068
[40]	training's amex_metric: 0.736963	valid_1's amex_metric: 0.730434
[50]	training's amex_metric: 0.744011	valid_1's amex_metric: 0.735555
[60]	training's amex_metric: 0.75012	valid_1's a

CPU times: user 7.12 s, sys: 660 ms, total: 7.78 s
Wall time: 1.78 s
--- y_pred ---
CPU times: user 59 s, sys: 35.6 s, total: 1min 34s
Wall time: 2min 3s
============== fold_id: 3
--- train ---
[LightGBM] [Info] Number of positive: 95063, number of negative: 272068
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.760389 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 158469
[LightGBM] [Info] Number of data points in the train set: 367131, number of used features: 581
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258935 -> initscore=-1.051512
[LightGBM] [Info] Start training from score -1.051512
Training until validation scores don't improve for 10 rounds
[10]	training's amex_metric: 0.708998	valid_1's amex_metric: 0.699921
[20]	training's amex_metric: 0.722115	valid_1's amex_metric: 0.713248
[30]	training's amex_metric: 0.730945	valid_1's amex_metric: 0.721164
[40]	training's amex_metric: 0.73

In [23]:
%%time
pd.DataFrame(oof_train).to_csv(f'work/oof_train_kfold_{book_name}.csv', index=False)

# for m in models:
#     print(m.best_score) 

scores = [
    m.best_score['valid_1']['amex_metric'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

===CV scores===
[0.7704307858267416, 0.7647541123020786, 0.7688835607684754, 0.7663499705421467, 0.7711977218574284]
0.7683232302593741
CPU times: user 1.05 s, sys: 158 ms, total: 1.2 s
Wall time: 1.49 s


In [24]:
from sklearn.metrics import accuracy_score

y_pred_oof = (oof_train > 0.5).astype(int)
print('accuracy_score:', accuracy_score(y_train, y_pred_oof))


accuracy_score: 0.8962788153745918


In [25]:
len(y_preds)

5

In [26]:
y_preds[0][:10]

array([0.03899969, 0.00162116, 0.02236095, 0.40752904, 0.80035568,
       0.00132009, 0.91103063, 0.38623696, 0.70653017, 0.00608877])

In [27]:
y_sub = (sum(y_preds) / len(y_preds)).round(2)
# y_sub = (y_sub > 0.5).astype(int)
print(y_sub[:10])
print('y_preds.shape', y_preds[1].shape)


[0.05 0.   0.03 0.42 0.75 0.   0.9  0.36 0.71 0.  ]
y_preds.shape (924621,)


In [28]:
sub = pd.read_csv('input/sample_submission.csv')
sub

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0


In [29]:
y_pred

array([0.04994909, 0.00250577, 0.02963214, ..., 0.67225442, 0.45180781,
       0.17143581])

In [30]:
y_sub

array([0.05, 0.  , 0.03, ..., 0.68, 0.46, 0.13])

In [31]:
# create submission
X_tiny = X_test[['count']]
X_tiny = pd.merge(X_tiny, sub, left_index=True, right_on='customer_ID')
X_tiny['prediction'] = y_sub
X_tiny = X_tiny.drop(columns='count')
sub = X_tiny

In [32]:
sub

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.05
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.00
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.03
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.42
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.75
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.01
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.64
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.68
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.46


In [33]:
sub.to_csv('output/submission/sub_plus_custom-metric.csv', index=False)
